In [ ]:
x_train_path = "/content/x_train.npy"
y_train_path = "/content/y_train.csv"
x_test_path = "/content/x_test.npy"
model_save_path = "/content/model.pth"
model_path = "/content/model.pth"
y_tmp_save_path = "/content/y_tmp.csv"
y_pred_save_path = "/content/y_pred.csv"

In [ ]:
!gdown

usage: gdown [-h] [-V] [-O OUTPUT] [-q] [--fuzzy] [--id] [--proxy PROXY] [--speed SPEED]
             [--no-cookies] [--no-check-certificate] [--continue] [--folder] [--remaining-ok]
             url_or_id
gdown: error: the following arguments are required: url_or_id


In [ ]:
file_id = "1Q4MQuo7_fcu2RIgIgzSH4GgFYDx5kvla"
url = f"https://drive.google.com/uc?id={file_id}"

In [ ]:
import os

# 다운로드 전의 파일 리스트를 가져옵니다.
before_files = set(os.listdir())

# gdown을 사용하여 파일을 다운로드합니다.
!gdown {url}

# 다운로드 후의 파일 리스트를 가져옵니다.
after_files = set(os.listdir())

# 다운로드된 파일명을 찾습니다.
downloaded_files = after_files - before_files
if downloaded_files:
    filename = downloaded_files.pop()
    print(f"Downloaded file: {filename}")
    downloaded_filepath = filename
else:
    print("No file downloaded.")

Downloading...
From: https://drive.google.com/uc?id=1Q4MQuo7_fcu2RIgIgzSH4GgFYDx5kvla
To: /content/ginseng_competition_data.zip
100% 391M/391M [00:08<00:00, 45.6MB/s]
No file downloaded.


In [ ]:
!unzip {downloaded_filepath}

unzip:  cannot find or open {downloaded_filepath}, {downloaded_filepath}.zip or {downloaded_filepath}.ZIP.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import numpy as np
import pandas as pd
import sys
import yaml

In [ ]:
# 데이터 전처리를 위한 변환 정의 (이미지 크기 조절과 채널 수 조절)
data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])
# 이미지 데이터를 담은 npy 파일 로드
image_data = np.load(x_train_path)
# 라벨 정보를 담은 CSV 파일 로드

labels_df = pd.read_csv(y_train_path)
# 'file_name' 열을 인덱스로 설정
labels_df.set_index('file_name', inplace=True)

# 이미지 파일과 라벨 정보를 매핑
labels = torch.tensor(labels_df['label'].values)

# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        target = self.targets[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample, target

# 데이터셋과 데이터 로더 생성
custom_dataset = CustomDataset(data=image_data, targets=labels, transform=data_transforms)
data_loader = DataLoader(dataset=custom_dataset, batch_size=32, shuffle=True)

# 간단한 신경망 모델 정의
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(16 * 32 * 32, 64)
        self.fc2 = nn.Linear(64, 3)  # 3개의 클래스 (high, mid, low)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16 * 32 * 32)  # Flatten the image tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델, 손실 함수, 최적화 함수 초기화
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
num_epochs = 10
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(data_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(data_loader)}], Loss: {loss.item():.4f}')

print('학습이 완료되었습니다.')

# 모델 저장
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')

Epoch [1/10], Step [10/10], Loss: 1.0639
Epoch [2/10], Step [10/10], Loss: 0.8649
Epoch [3/10], Step [10/10], Loss: 0.8369
Epoch [4/10], Step [10/10], Loss: 0.6025
Epoch [5/10], Step [10/10], Loss: 0.8077
Epoch [6/10], Step [10/10], Loss: 0.4444
Epoch [7/10], Step [10/10], Loss: 0.3404
Epoch [8/10], Step [10/10], Loss: 0.2418
Epoch [9/10], Step [10/10], Loss: 0.2242
Epoch [10/10], Step [10/10], Loss: 0.1504
학습이 완료되었습니다.
Model saved to /content/model.pth


In [ ]:
# 모델 정의
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(16 * 32 * 32, 64)
        self.fc2 = nn.Linear(64, 3)  # 3개의 클래스 (high, mid, low)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16 * 32 * 32)  # Flatten the image tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 불러오기
model = SimpleNet()
model.load_state_dict(torch.load(model_path))
model.eval()

# 데이터 전처리 변환 정의 (이미지 크기 조절과 채널 수 조절)
data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# 테스트 데이터를 담은 npy 파일 로드
test_image_data = np.load(x_test_path)

# 테스트 데이터에 대한 추론 수행
results = []
for image_data in test_image_data:
    # 이미지 전처리
    image = data_transforms(image_data)
    image = image.unsqueeze(0)  # 배치 차원을 추가
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        results.append(predicted.item())

# 결과를 DataFrame으로 저장
test_df = pd.DataFrame({'file_name': [f'test_{i}.jpg' for i in range(len(results))], 'label': results})

# 결과를 CSV 파일로 저장
test_df.to_csv(y_tmp_save_path, index=False)

print('테스트 결과를 CSV 파일로 저장했습니다.')


테스트 결과를 CSV 파일로 저장했습니다.


In [ ]:
import csv


# 원본 CSV 파일 열기
with open(y_tmp_save_path, 'r', newline='') as infile, \
     open(y_pred_save_path, 'w', newline='') as outfile:

    # csv.reader와 csv.writer 객체 생성
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # 첫 번째 행(헤더) 건너뛰기
    next(reader)

    # 각 행에서 첫 번째 열 제거하고 새 파일에 쓰기
    for row in reader:
        writer.writerow(row[1:])  # 첫 번째 열을 제외한 나머지 열을 쓴다

print(f'The modified data has been saved to {y_pred_save_path}')


The modified data has been saved to /content/y_pred.csv


In [ ]:
#이제 생성된 y_pred 파일을 가지고 채점을 해보세요!

In [ ]:
#!pip -q install gradio==3.45.0

In [ ]:
import gradio as gr
import io

In [ ]:
# 모델을 불러오는 함수 정의
def load_model(model_path):
    model = SimpleNet()  # SimpleNet은 이미 정의된 모델 클래스여야 합니다.
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# 이미지를 처리하고 예측을 수행하는 함수 정의
def predict_image(image):
    # 이미지를 모델이 요구하는 형태로 변환
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor()
    ])

    image = Image.open(io.BytesIO(image)).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # 모델은 배치로 입력을 받기 때문에 차원 추가 필요

    # 예측 수행
    with torch.no_grad():
        prediction = model(image)
        _, predicted_class = torch.max(prediction, 1)
        return predicted_class.item()

# 모델 경로 설정
model = load_model(model_path)

# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=predict_image,  # 실행할 함수
    inputs=gr.inputs.Image(shape=(64, 64)),  # 입력 형태
    outputs="text",  # 출력 형태
    title="Image Classifier",  # 인터페이스 제목
    description="Upload an image to classify."  # 인터페이스 설명
)

# 인터페이스 실행
iface.launch()

<ipython-input-16-4d346478745a>:32: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(shape=(64, 64)),  # 입력 형태
<ipython-input-16-4d346478745a>:32: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(shape=(64, 64)),  # 입력 형태


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>